In [2]:
%pip install --quiet transformers==4.37.2 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2 datasets==2.14.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 13.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.nn.functional as F
import peft
import transformers
from datasets import load_dataset
import random
const_seed = 100

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Part 0: Initializing the model and tokenizer

In [5]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.4 MB/s eta 0:00:00
  Created wheel for flash-attn: filename=flash_attn-2.5.5-cp310-cp310-linux_x86_64.whl size=120352304 sha256=e70f2f0d4fae98c9ff9742404f3e067bbae56bd3212578ff81c30f73c5ae1a15
  Stored in directory: /root/.cache/pip/wheels/b2/67/52/8b6d5fcffdd9e1ec868f554cdef8f03eedb4bf4dcac852fca2
Successfully built flash-attn


let's take mistral model for our experiments (https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2) that was tuned to follow user instructions. Pay attention that we load model in 4 bit to decrease the memory usage.

model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token_id = tokenizer.eos_token_id


model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.2", device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float16,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()
model.enable_input_require_grads()
# more on gradient checkpointing: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

# Part 1 (5 points): Prompt-engineering

**There are different strategies for text generation in huggingface:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [8]:
def get_answer(tokenizer, model,messages, max_new_tokens=200,temperature=0.5, do_sample=True):
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)
    generated_ids = model.generate(model_inputs,max_new_tokens=max_new_tokens,temperature=temperature,do_sample=do_sample)
    decoded = tokenizer.batch_decode(generated_ids,skip_special_tokens=True)
    return decoded

In [9]:
# Let's try our model
messages = [
    {"role": "user", "content": "Write an explanation of tensors for 5 year old"},
]

print(get_answer(tokenizer, model, messages)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


[INST] Write an explanation of tensors for 5 year old [/INST] Tensors are like magical boxes that can hold different things, but in a very special way. Imagine you have a crayon box, it can hold many crayons of different colors. Now, imagine a box that can hold not only crayons but also numbers, shapes, and pictures. This kind of box is called a tensor.

Tensors can help us understand and describe things in the world around us, like the shape of a ball, the colors of a rainbow, or the number of apples in a basket. They make it easier for us to solve problems and learn new things! So, just like how a crayon box can help us draw beautiful pictures, tensors can help us understand and explore the world in a more magical way.


You should obtain an explanation from the model. If so, let us go further!

Now we will take a sample from boolQ (https://huggingface.co/datasets/google/boolq) dataset and try prompting techniques to extract the needed answer and calculate its quality

In [10]:
df = load_dataset("google/boolq")
df

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})

In [11]:
# Fixing 20 validation examples
random.seed(const_seed)
idx = random.sample(range(1, 3270), 20)

In [12]:
# sample you will work with
df_sample = df["validation"].select(idx)

In [74]:
# For instance, you can construct your prompt the following way
answers = []
for i in range(0, 20):
    messages = [
        {
            "role": "user",
            "content": f'''MY QUESTION is ({df_sample['question'][i]}) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)
                    format:
                        text： {df_sample['question'][i]}（Repeating my question here）
                        answer:
                      '''
        }
    ]
    print(f"message:{messages}")
    answer=get_answer(tokenizer, model, messages)[0]
    print(answer)
    answers.append(answer)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


message:[{'role': 'user', 'content': "MY QUESTION is (is the vice president the head of the senate) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)\n                    format:\n                        text： is the vice president the head of the senate（Repeating my question here）\n                        answer:\n                      "}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] MY QUESTION is (is the vice president the head of the senate) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)
                    format:
                        text： is the vice president the head of the senate（Repeating my question here）
                        answer:
                       [/INST] False. The Vice President serves as the President of the Senate in a largely ceremonial role, presiding over the Senate and casting tie-breaking votes when necessary. However, the Vice President does not hold the legislative power or authority of the Senate as a whole. The Senate's leadership and day-to-day operations are carried out by the Senate Majority and Minority Leaders and their respective parties.
message:[{'role': 'user', 'content': "MY QUESTION is (can i get $1 000 bill from the bank) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)\n   

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

[INST] MY QUESTION is (can i have multiple foreign keys in a table) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)
                    format:
                        text： can i have multiple foreign keys in a table（Repeating my question here）
                        answer:
                       [/INST] True. A table in a relational database can have multiple foreign keys, each referencing a primary key in a different table.
message:[{'role': 'user', 'content': "MY QUESTION is (is there a delay of game in hockey) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)\n                    format:\n                        text： is there a delay of game in hockey（Repeating my question here）\n                        answer:\n                      "}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] MY QUESTION is (is there a delay of game in hockey) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)
                    format:
                        text： is there a delay of game in hockey（Repeating my question here）
                        answer:
                       [/INST] True.

In hockey, there is a rule called "delay of game," which penalizes players for actions that excessively delay the game. This can include things like shooting the puck over the glass or holding the puck in the defensive zone for too long.
message:[{'role': 'user', 'content': "MY QUESTION is (do you need a permit to carry a gun in nh) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)\n                    format:\n                        text： do you need a permit to carry a gun in nh（Repeating my question here）\n                        answer:\n                   

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] MY QUESTION is (do you need a permit to carry a gun in nh) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)
                    format:
                        text： do you need a permit to carry a gun in nh（Repeating my question here）
                        answer:
                       [/INST] True. In New Hampshire, you do need a permit to carry a concealed gun. Open carry does not require a permit. However, for the purpose of your question, the requirement for a permit applies to carrying a gun.
message:[{'role': 'user', 'content': "MY QUESTION is (is los lobos in the rock and roll hall of fame) generate True or False based on MY QUESTION ,just need True or False(I don't need any other answer like yes or NO.)\n                    format:\n                        text： is los lobos in the rock and roll hall of fame（Repeating my question here）\n                        answer:\n                      "}]
[INST]

In [78]:
post_ = []
pattern_inst = re.compile(r'INST', re.IGNORECASE)
pattern_tf = re.compile(r'\b(true|false)\b', re.IGNORECASE)

for answer_text in answers:
    # Find all occurrences of "INST"
    inst_matches = [match for match in pattern_inst.finditer(answer_text)]
    # Find all occurrences of "true" or "false"
    tf_matches = [match for match in pattern_tf.finditer(answer_text)]

    if len(inst_matches) >= 2 and tf_matches:
        # Get the end position of the second "INST"
        second_inst_end_pos = inst_matches[1].end()

        # Search for "true" or "false" after the second "INST"
        match_tf = pattern_tf.search(answer_text, second_inst_end_pos)

        if match_tf:
            # If found after the second "INST", append this match
            post.append(match_tf.group(1))
        else:
            # If not found after the second "INST", append the last occurrence of "true" or "false"
            post.append(tf_matches[-1].group(1))
    elif tf_matches:
        # If "INST" is not found or less than 2, but "true" or "false" is found, append the last occurrence
        post.append(tf_matches[-1].group(1))
    else:
        # If neither "INST" nor "true" or "false" are found
        post.append("未找到")

print(post)



['False', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'true', 'True', 'False', 'true', 'True', 'False', 'False', 'True', 'True', 'True', 'False', 'False', 'False', 'False', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'true', 'True', 'False', 'true', 'True', 'False', 'False', 'True', 'True', 'true', 'False', 'False', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'true', 'True', 'False', 'true', 'True', 'False', 'False', 'True', 'True', 'true', 'False', 'False', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'True', 'True', 'False', 'True', 'true', 'False', 'False', 'True', 'True', 'True', 'False', 'False', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'True', 'True', 'False', 'True', 'true', 'False', 'False', 'True', 'True', 'True', 'False']


In [79]:
bool_pre=[]
for i in range(0,20):
  prediction = post[i].lower() == 'true'
  bool_pre.append(prediction)
bool_pre

[False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 False]

Is anything wrong with the output? Now it is time for you to play around and try to come up with some better prompt.

In [80]:
# TODO: create function to evaluate answers
# Note: you can adapt function for different answer structures,
# but you should be able to automatically extract the target "true" or "false" components

def evaluate_answers(true_answers, predictions):

    assert len(true_answers) == len(predictions)

    total_questions = len(true_answers)
    matched_count = 0

    for true_answer, prediction in zip(true_answers, predictions):

        if true_answer == prediction:
            matched_count += 1

    score= (matched_count / 20) * 100

    return score
print(f"naive prompt:{evaluate_answers(df_sample['answer'], bool_pre)}")

naive prompt:55.00000000000001


TODO: Try and compare "naive" prompting (your best hand-crafted variant), few-shot prompting (https://www.promptingguide.ai/techniques/fewshot) and chain-of-thought prompting (step-be-step thinking - https://www.promptingguide.ai/techniques/cot).

Save the generation results into separate csv files and do not forget to attach them to your homework.

Naive prompting

In [81]:
import pandas as pd
naive_p = pd.DataFrame({"answers": answers})
naive_p.to_csv("answer_n.csv", index=False)

Few-Shot Prompting

In [86]:
answers_F = []

idy=random.sample(range(1, 3270), 4)

for i in range(0, 20):
    messages_F = [
        {
            "role": "user",
            "content": f'''Answer the fifth question with true or false
            1. Question: {df["validation"][idy[0]]['question']}
              Answer: {df["validation"][idy[0]]['answer']}
            2. Question: {df["validation"][idy[1]]['question']}
              Answer: {df["validation"][idy[1]]['answer']}
            3. Question: {df["validation"][idy[2]]['question']}
              Answer: {df["validation"][idy[2]]['answer']}
            4. Question: {df["validation"][idy[3]]['question']}
              Answer: {df["validation"][idy[3]]['answer']}
            5. Question: {df_sample['question'][i]}
              Answer: (true/false)
            '''
        }
    ]
    print(messages_F)
    answersd=get_answer(tokenizer,model,messages_F)[0]
    print(answersd)
    answers_F.append(answersd)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'role': 'user', 'content': 'Answer the fifth question with true or false\n            1. Question: did ireland ever win the soccer world cup\n              Answer: False\n            2. Question: is there always a way to win solitaire\n              Answer: False\n            3. Question: is a fire 7 the same as a kindle\n              Answer: True\n            4. Question: do ncaa division 2 schools offer athletic scholarships\n              Answer: False\n            5. Question: is the vice president the head of the senate\n              Answer: (true/false)\n            '}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Answer the fifth question with true or false
            1. Question: did ireland ever win the soccer world cup
              Answer: False
            2. Question: is there always a way to win solitaire
              Answer: False
            3. Question: is a fire 7 the same as a kindle
              Answer: True
            4. Question: do ncaa division 2 schools offer athletic scholarships
              Answer: False
            5. Question: is the vice president the head of the senate
              Answer: (true/false)
             [/INST] The Vice President is not the head of the Senate. The Vice President serves as the President of the Senate, but only has a vote in the event of a tie. The Senate is presided over by the Senate President or, in the absence of the President, by the Senate Pro Tempore.
[{'role': 'user', 'content': 'Answer the fifth question with true or false\n            1. Question: did ireland ever win the soccer world cup\n              Answer: False\n  

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

[INST] Answer the fifth question with true or false
            1. Question: did ireland ever win the soccer world cup
              Answer: False
            2. Question: is there always a way to win solitaire
              Answer: False
            3. Question: is a fire 7 the same as a kindle
              Answer: True
            4. Question: do ncaa division 2 schools offer athletic scholarships
              Answer: False
            5. Question: can i have multiple foreign keys in a table
              Answer: (true/false)
             [/INST] Answer: True. In a relational database, you can have multiple foreign keys in a single table, each referencing a primary key in a different table. This is used to establish relationships between multiple entities in the database.
[{'role': 'user', 'content': 'Answer the fifth question with true or false\n            1. Question: did ireland ever win the soccer world cup\n              Answer: False\n            2. Question: is there always

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Answer the fifth question with true or false
            1. Question: did ireland ever win the soccer world cup
              Answer: False
            2. Question: is there always a way to win solitaire
              Answer: False
            3. Question: is a fire 7 the same as a kindle
              Answer: True
            4. Question: do ncaa division 2 schools offer athletic scholarships
              Answer: False
            5. Question: is there a delay of game in hockey
              Answer: (true/false)
             [/INST] Answer:
Yes, the answer for question 5 is "True". In hockey, a delay of game penalty can be called when a team fails to keep the puck in their own defensive zone for a sufficient length of time or when they shoot the puck out of their own defensive zone over the boards. This results in a faceoff in the offensive zone for the opposing team.
[{'role': 'user', 'content': 'Answer the fifth question with true or false\n            1. Question: did irela

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Answer the fifth question with true or false
            1. Question: did ireland ever win the soccer world cup
              Answer: False
            2. Question: is there always a way to win solitaire
              Answer: False
            3. Question: is a fire 7 the same as a kindle
              Answer: True
            4. Question: do ncaa division 2 schools offer athletic scholarships
              Answer: False
            5. Question: do you need a permit to carry a gun in nh
              Answer: (true/false)
             [/INST] Answer: True, in the state of New Hampshire, you do need a permit to carry a gun. However, it's important to note that specific laws and requirements may vary, so it's always a good idea to check with local authorities or consult legal resources for the most accurate information.
[{'role': 'user', 'content': 'Answer the fifth question with true or false\n            1. Question: did ireland ever win the soccer world cup\n              Answer

In [91]:
answers_F[0]

'[INST] Answer the fifth question with true or false\n            1. Question: did ireland ever win the soccer world cup\n              Answer: False\n            2. Question: is there always a way to win solitaire\n              Answer: False\n            3. Question: is a fire 7 the same as a kindle\n              Answer: True\n            4. Question: do ncaa division 2 schools offer athletic scholarships\n              Answer: False\n            5. Question: is the vice president the head of the senate\n              Answer: (true/false)\n             [/INST] The Vice President is not the head of the Senate. The Vice President serves as the President of the Senate, but only has a vote in the event of a tie. The Senate is presided over by the Senate President or, in the absence of the President, by the Senate Pro Tempore.'

In [92]:
import re

post_F = []
pattern_inst = re.compile(r'INST', re.IGNORECASE)
pattern_tf = re.compile(r'\b(true|false)\b', re.IGNORECASE)

for answer_text in answers_F:
    # Find all occurrences of "INST"
    inst_matches = [match for match in pattern_inst.finditer(answer_text)]
    # Find all occurrences of "true" or "false"
    tf_matches = [match for match in pattern_tf.finditer(answer_text)]

    if len(inst_matches) >= 2 and tf_matches:
        # Get the end position of the second "INST"
        second_inst_end_pos = inst_matches[1].end()

        # Search for "true" or "false" after the second "INST"
        match_tf = pattern_tf.search(answer_text, second_inst_end_pos)

        if match_tf:
            # If found after the second "INST", append this match
            post_F.append(match_tf.group(1))
        else:
            # If not found after the second "INST", append the last occurrence of "true" or "false"
            post_F.append(tf_matches[-1].group(1))
    elif tf_matches:
        # If "INST" is not found or less than 2, but "true" or "false" is found, append the last occurrence
        post_F.append(tf_matches[-1].group(1))
    else:
        # If neither "INST" nor "true" or "false" are found
        post_F.append("未找到")

print(post_F)


['false', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'False']


In [93]:
bool_pre=[]
for i in range(0,20):
  prediction = post_F[i].lower() == 'true'
  bool_pre.append(prediction)
bool_pre

[False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False]

In [94]:
def evaluate_answers(true_answers, predictions):

    assert len(true_answers) == len(predictions)

    total_questions = len(true_answers)
    matched_count = 0

    for true_answer, prediction in zip(true_answers, predictions):

        if true_answer == prediction:
            matched_count += 1

    score= (matched_count / 20) * 100

    return score
print(evaluate_answers(df_sample['answer'], bool_pre))

70.0


In [95]:
import pandas as pd
f = pd.DataFrame({"answers_F": answers_F})
f.to_csv("answers_F.csv", index=False)

Chain-of-Thought Prompting

In [34]:
answers_C = []

idy=random.sample(range(1, 3270), 4)

for i in range(0, 20):
    messages_C = [
        {
            "role": "user",
            "content": f'''Answer the forth question with true or false,Answer format must be:Answer 4 is (True or False)
            1. Question: {df["validation"][idy[0]]['question']}
              Answer: {df["validation"][idy[0]]['answer']},Because:{df["validation"][idy[0]]['passage']}
            2. Question: {df["validation"][idy[1]]['question']}
              Answer: {df["validation"][idy[1]]['answer']},Because:{df["validation"][idy[1]]['passage']}
            3. Question: {df["validation"][idy[2]]['question']}
              Answer: {df["validation"][idy[2]]['answer']},Because:{df["validation"][idy[2]]['passage']}
            4. Question: {df_sample['question'][i]}
              Answer: (true/false)
            '''
        }
    ]
    print(messages_C)
    answersc=get_answer(tokenizer,model,messages_C)[0]
    print(answersc)
    answers_C.append(answersc)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'role': 'user', 'content': "Answer the forth question with true or false,Answer format must be:Answer 4 is (True or False)\n            1. Question: does every state have a good samaritan law\n              Answer: True,Because:All 50 states and the District of Columbia have some type of Good Samaritan law. The details of good Samaritan laws/acts vary by jurisdiction, including who is protected from liability and under what circumstances.\n            2. Question: can picking your nose affect your sense of smell\n              Answer: False,Because:Picking one's nose with dirty fingers or fingernails may increase risks of infection that may include an increase in the diversity of nose flora (and thus infection or illness), or occasional nosebleeds. One case of rhinotillexomania resulted in perforation of the nasal septum and self-induced ethmoidectomy. Nose picking, however, should not affect the sense of smell, as the nasal cavity where the olfactory nerves are located is too high u

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Answer the forth question with true or false,Answer format must be:Answer 4 is (True or False)
            1. Question: does every state have a good samaritan law
              Answer: True,Because:All 50 states and the District of Columbia have some type of Good Samaritan law. The details of good Samaritan laws/acts vary by jurisdiction, including who is protected from liability and under what circumstances.
            2. Question: can picking your nose affect your sense of smell
              Answer: False,Because:Picking one's nose with dirty fingers or fingernails may increase risks of infection that may include an increase in the diversity of nose flora (and thus infection or illness), or occasional nosebleeds. One case of rhinotillexomania resulted in perforation of the nasal septum and self-induced ethmoidectomy. Nose picking, however, should not affect the sense of smell, as the nasal cavity where the olfactory nerves are located is too high up to reach.
            3. 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

[INST] Answer the forth question with true or false,Answer format must be:Answer 4 is (True or False)
            1. Question: does every state have a good samaritan law
              Answer: True,Because:All 50 states and the District of Columbia have some type of Good Samaritan law. The details of good Samaritan laws/acts vary by jurisdiction, including who is protected from liability and under what circumstances.
            2. Question: can picking your nose affect your sense of smell
              Answer: False,Because:Picking one's nose with dirty fingers or fingernails may increase risks of infection that may include an increase in the diversity of nose flora (and thus infection or illness), or occasional nosebleeds. One case of rhinotillexomania resulted in perforation of the nasal septum and self-induced ethmoidectomy. Nose picking, however, should not affect the sense of smell, as the nasal cavity where the olfactory nerves are located is too high up to reach.
            3. 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Answer the forth question with true or false,Answer format must be:Answer 4 is (True or False)
            1. Question: does every state have a good samaritan law
              Answer: True,Because:All 50 states and the District of Columbia have some type of Good Samaritan law. The details of good Samaritan laws/acts vary by jurisdiction, including who is protected from liability and under what circumstances.
            2. Question: can picking your nose affect your sense of smell
              Answer: False,Because:Picking one's nose with dirty fingers or fingernails may increase risks of infection that may include an increase in the diversity of nose flora (and thus infection or illness), or occasional nosebleeds. One case of rhinotillexomania resulted in perforation of the nasal septum and self-induced ethmoidectomy. Nose picking, however, should not affect the sense of smell, as the nasal cavity where the olfactory nerves are located is too high up to reach.
            3. 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Answer the forth question with true or false,Answer format must be:Answer 4 is (True or False)
            1. Question: does every state have a good samaritan law
              Answer: True,Because:All 50 states and the District of Columbia have some type of Good Samaritan law. The details of good Samaritan laws/acts vary by jurisdiction, including who is protected from liability and under what circumstances.
            2. Question: can picking your nose affect your sense of smell
              Answer: False,Because:Picking one's nose with dirty fingers or fingernails may increase risks of infection that may include an increase in the diversity of nose flora (and thus infection or illness), or occasional nosebleeds. One case of rhinotillexomania resulted in perforation of the nasal septum and self-induced ethmoidectomy. Nose picking, however, should not affect the sense of smell, as the nasal cavity where the olfactory nerves are located is too high up to reach.
            3. 

In [35]:
import re

post_C = []
pattern_inst = re.compile(r'INST', re.IGNORECASE)
pattern_tf = re.compile(r'\b(true|false)\b', re.IGNORECASE)

for answer_text in answers_C:
    # Find all occurrences of "INST"
    inst_matches = [match for match in pattern_inst.finditer(answer_text)]
    # Find all occurrences of "true" or "false"
    tf_matches = [match for match in pattern_tf.finditer(answer_text)]

    if len(inst_matches) >= 2 and tf_matches:
        # Get the end position of the second "INST"
        second_inst_end_pos = inst_matches[1].end()

        # Search for "true" or "false" after the second "INST"
        match_tf = pattern_tf.search(answer_text, second_inst_end_pos)

        if match_tf:
            # If found after the second "INST", append this match
            post_C.append(match_tf.group(1))
        else:
            # If not found after the second "INST", append the last occurrence of "true" or "false"
            post_C.append(tf_matches[-1].group(1))
    elif tf_matches:
        # If "INST" is not found or less than 2, but "true" or "false" is found, append the last occurrence
        post_C.append(tf_matches[-1].group(1))
    else:
        # If neither "INST" nor "true" or "false" are found
        post_C.append("未找到")

print(post_C)

['False', 'False', 'True', 'False', 'True', 'True', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'False']


In [36]:
bool_pre=[]
for i in range(0,20):
  prediction = post_C[i].lower() == 'true'
  bool_pre.append(prediction)
bool_pre

[False,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False]

In [37]:
def evaluate_answers(true_answers, predictions):

    assert len(true_answers) == len(predictions)

    total_questions = len(true_answers)
    matched_count = 0

    for true_answer, prediction in zip(true_answers, predictions):

        if true_answer == prediction:
            matched_count += 1

    score= (matched_count / 20) * 100

    return score
print(evaluate_answers(df_sample['answer'], bool_pre))

65.0


In [38]:
import pandas as pd
C = pd.DataFrame({"answers_C": answers_C})
C.to_csv("answers_C.csv", index=False)

# Part 2 (5 points): Fine-tuning with PEFT and LoRA

TODO: initialize Trainer and pass train part of our dataset for 2-3 epoches

Note: carefully set max_seq_length and args (that are transformers.TrainingArguments)

TODO: save and check your tuned model. Provide scores on our 20 validation examples and save result to csv file